In [1]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.4/491.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 13.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system ==

# предобработка данных

In [3]:
import pandas as pd
import re

df_150 = pd.read_csv("example150.csv", encoding='utf-8')

replacement_dict = {
    # Признание вины и раскаяние
    "признание вины": "признание вины",
    "признание своей вины": "признание вины",
    "полное признание вины": "признание вины",
    "полное признание своей вины": "признание вины",
    "чистосердечное признание вины": "признание вины",
    "чистосердечное признание": "признание вины",
    "полное признание подсудимым своей вины": "признание вины",
    "полное признание им своей вины": "признание вины",
    "признает признание вины": "признание вины",
    "раскаяние": "раскаяние",
    "раскаяние в содеянном": "раскаяние",

    # Дети и иждивенцы
    "наличие малолетнего ребенка": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие на иждивении малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие ребенка на иждивении": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие двоих малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие троих малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие на иждивении несовершеннолетнего ребенка": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие на иждивении троих малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие 1 несовершеннолетнего ребенка": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие 2 несовершеннолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие 3 малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие малолетнего ребёнка": "наличие несовершеннолетнего ребенка на иждивении",
    "нахождение на иждивении несовершеннолетнего ребёнка": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие на иждивении четверых малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие на иждивении пятерых детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие двоих несовершеннолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "нахождение на иждивении двоих малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие несовершеннолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие малолетнего ребенка на иждивении": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие трех малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие на иждивении двоих малолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие на иждивении несовершеннолетних детей": "наличие несовершеннолетнего ребенка на иждивении",
    "наличие на иждивении малолетнего ребенка": "наличие несовершеннолетнего ребенка на иждивении",


    # Состояние здоровья
    "наличие заболевания": "состояние здоровья",
    "наличие тяжелых заболеваний": "состояние здоровья",
    "инвалидность": "состояние здоровья",
    "наличие хронического заболевания": "состояние здоровья",
    "диагностированные у неё заболевания": "состояние здоровья",
    "диагностированное у него заболевание": "состояние здоровья",
    "наличие травмы и заболеваний": "состояние здоровья",
    "наличие психического расстройства": "состояние здоровья",
    "заболевание": "состояние здоровья",
    "третья группа инвалидности": "состояние здоровья",
    "состояние здоровья подсудимого": "состояние здоровья",
    "состояние здоровья сына": "состояние здоровья",
    "состояние здоровья родственников": "состояние здоровья",
    "состояние здоровья членов семьи": "состояние здоровья",
    "состояние здоровья матери": "состояние здоровья",
    "состояние здоровья супруги": "состояние здоровья",
    "наличие хронических заболеваний": "состояние здоровья",
    "инвалидность 3 группы бессрочно": "состояние здоровья",
    "инвалидности у близких родственников": "состояние здоровья",

    # Характеристики
    "удовлетворительная характеристика по месту жительства": "положительная характеристика",
    "положительные характеристики по месту жительства": "положительная характеристика",
    "положительную характеристику его личности": "положительная характеристика",
    "положительная характеристика по месту работы": "положительная характеристика",
    "положительные характеристики с места жительства": "положительная характеристика",
    "положительную характеристику с места жительства": "положительная характеристика",
    "положительную характеристику": "положительная характеристика",
    "положительная характеристика": "положительная характеристика",
    "положительные характеристики": "положительная характеристика",
    "характеризуется положительно": "положительная характеристика",
    "положительную характеристику её личности": "положительная характеристика",
    "положительная характеристика и работы": "положительная характеристика",
    "удовлетворительную характеристику его личности": "удовлетворительная характеристика",
    "положительную характеристику личности": "положительная характеристика",
    "положительные характеристики по месту жительства и работы" : "положительная характеристика",

    # Активность в расследовании
    "активное способствование раскрытию и расследованию преступления": "активное способствование раскрытию и расследованию преступления",
    "активное способствование раскрытию преступления": "активное способствование раскрытию и расследованию преступления",
    "активное способствование расследованию преступления": "активное способствование раскрытию и расследованию преступления",
    "активно способствовал раскрытию и расследованию преступлени": "активное способствование раскрытию и расследованию преступления",
    "cпособствование раскрытию и расследованию преступления": "активное способствование раскрытию и расследованию преступления",
    "активное способствование расследованию": "активное способствование раскрытию и расследованию преступления",
    "активное способствование раскрытию": "активное способствование раскрытию и расследованию преступления",
    "изобличение других соучастников преступления": "активное способствование раскрытию и расследованию преступления",
    "дача правдивых показаний": "активное способствование раскрытию и расследованию преступления",
    "дал объяснения об обстоятельствах преступления": "активное способствование раскрытию и расследованию преступления",
    "активное способствование раскрытию и расследованию": "активное способствование раскрытию и расследованию преступления",

    # Явка с повинной
    "явка с повинной": "явка с повинной",
    "явку с повинной": "явка с повинной",

    # Первая судимость
    "привлечение к уголовной ответственности впервые": "первая судимость",
    "ранее к уголовной ответственности не привлекалась": "первая судимость",
    "ранее не судим": "первая судимость",
    "совершение преступления впервые": "первая судимость",
    "ранее к уголовной ответственности не привлекался": "первая судимость",

    # Возраст
    "молодой возраст": "возраст",

    # Прочие обстоятельства
    "беременность сожительницы": "беременность сожительницы",
    "беременность гражданской супруги":"беременность сожительницы",
    "инвалидность матери": "наличие иждивенцев",
    "наличие престарелых родителей": "наличие иждивенцев",
    "наличие пожилых родителей": "наличие иждивенцев",
    "наличие на его иждивении престарелой бабушки": "наличие иждивенцев",
    "наличие матери-инвалида и престарелой бабушки": "наличие иждивенцев",
    "наличие пожилой матери": "наличие иждивенцев",
    "нахождение на иждивении супруги, находящееся в состоянии беременности": "беременность сожительницы",
    "добровольное возмещение ущерба причиненного потерпевшей": "возмещение ущерба",
    "добровольное возмещение имущественного ущерба потерпевшей": "возмещение ущерба",
    "принесение публичных извинений": "публичные извинения",
    "занятие общественно-полезным трудом": "общественно-полезный труд",
    "наличие грамот и благодарственных писем": "поощрения",
    "поведение после совершенного преступления": "хорошее поведение",
    "на учете у нарколога и психиатра не состоит": "отсутствие учетов",
    "участие в Специальной военной операции": "участие в СВО",
    "участие подсудимого в Специальной военной операции": "участие в СВО",
    "получение ранения в ходе боевых действий": "ранение в СВО",
    "установленные медицинским освидетельствованием РВК": "медосвидетельствование",
    "аличество на иждивении двоих малолетних детей": "наличие несовершеннолетнего ребенка",
    "наличие на иждивении тетки пенсионера инвалида": "наличие иждивенцев",
    "активное способствование раскрытию и расследованию преступленияо": "активное способствование раскрытию и расследованию преступления" # Обработка артефакта
}

def standardize_text(text, replacement_dict):
    if pd.isna(text):
        return text

    text = re.sub(r'[;,]', ';', text)
    text = re.sub(r'\s+', ' ', text).strip().lower()

    for old, new in replacement_dict.items():
        text = re.sub(r'\b' + re.escape(old.lower()) + r'\b', new, text)

    factors = list(set(text.split(';')))
    return '; '.join(factors).strip('; ')

df_150['mitigating_factors_standardized'] = df_150['mitigating_factors'].apply(lambda x: standardize_text(x, replacement_dict))
df_150['result_text'] = df_150['result_text'].apply(lambda x: standardize_text(x, replacement_dict))
df_150.to_csv('example150_with_mitigating_factor_standardized.csv', index=False, encoding='utf-8')

In [3]:
# random sample 10k
df_0k1k = pd.read_csv('cases_random_0kto1k.csv')
df_0k1k['result_text'] = df_0k1k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

df_1k2k = pd.read_csv('cases_random_1kto2k.csv')
df_1k2k['result_text'] = df_1k2k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

df_2k3k = pd.read_csv('cases_random_2kto3k.csv')
df_2k3k['result_text'] = df_2k3k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

df_3k4k = pd.read_csv('cases_random_3kto4k.csv')
df_3k4k['result_text'] = df_3k4k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

df_4k5k = pd.read_csv('cases_random_4kto5k.csv')
df_4k5k['result_text'] = df_4k5k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

df_5k6k = pd.read_csv('cases_random_5kto6k.csv')
df_5k6k['result_text'] = df_5k6k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

df_6k7k = pd.read_csv('cases_random_6kto7k.csv')
df_6k7k['result_text'] = df_6k7k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

df_7k8k = pd.read_csv('cases_random_7kto8k.csv')
df_7k8k['result_text'] = df_7k8k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

df_8k9k = pd.read_csv('cases_random_8kto9k.csv')
df_8k9k['result_text'] = df_8k9k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

df_9k10k = pd.read_csv('cases_random_9kto10k.csv')
df_9k10k['result_text'] = df_9k10k['result_text'].apply(lambda x: standardize_text(x, replacement_dict))

# подключение модели

In [4]:
# это чтобы не подключаться к WabdB и чтобы он не беспокоил просьбами об апи ключе
import os
os.environ["WANDB_DISABLED"] = "true"

In [5]:
import pandas as pd
import torch
from datasets import Dataset
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments

labels_list = [
    # "признание вины", "раскаяние",
    "наличие несовершеннолетнего ребенка на иждивении",
    "состояние здоровья", "положительная характеристика", "удовлетворительная характеристика",
    "активное способствование раскрытию и расследованию преступления", "явка с повинной",
    "первая судимость", "возраст", "беременность сожительницы", "наличие иждивенцев",
    "возмещение ущерба", "публичные извинения", "общественно-полезный труд", "поощрения",
    "хорошее поведение", "отсутствие учетов", "участие в СВО", "ранение в СВО", "медосвидетельствование"
]

label2id = {label: idx for idx, label in enumerate(labels_list)}

tokenizer = BertTokenizer.from_pretrained('sberbank-ai/ruBert-base')
model = BertForSequenceClassification.from_pretrained('sberbank-ai/ruBert-base', num_labels=len(labels_list), problem_type="multi_label_classification")

# - 'result_text' — текст приговора
# - 'mitigating_factors_standardized' — список обстоятельств (как list или строка через ; )

def encode_labels(row):
    labels = [0] * len(labels_list)
    if isinstance(row['mitigating_factors_standardized'], str):
        factors = [f.strip() for f in row['mitigating_factors_standardized'].split(';')]
    else:
        factors = row['mitigating_factors_standardized']
    for f in factors:
        if f in label2id:
            labels[label2id[f]] = 1
    return labels

def tokenize_and_encode(example):
    inputs = tokenizer(example['result_text'], padding="max_length", truncation=True, max_length=512)
    labels = encode_labels(example)
    labels = [float(x) for x in labels]
    inputs['labels'] = labels
    return inputs


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.txt:   0%|          | 0.00/1.78M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/590 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/716M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sberbank-ai/ruBert-base and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Map:   0%|          | 0/149 [00:00<?, ? examples/s]

In [24]:
# df_train = df_150
excluded = {'признание вины', 'раскаяние'}

def filter_factors(factors):
    if isinstance(factors, str):
        factors = [f.strip() for f in factors.split(';')]
    return [f for f in factors if f not in excluded]

df_train['factors_filtered'] = df_train['mitigating_factors_standardized'].apply(filter_factors)
df_big['pred_filtered'] = df_big['predicted_mitigating_factors'].apply(filter_factors)
df_big['true_filtered'] = df_big['mitigating_factors_standardized'].apply(filter_factors)


dataset = Dataset.from_pandas(df_train)
dataset = dataset.map(tokenize_and_encode, remove_columns=dataset.column_names)

Map:   0%|          | 0/149 [00:00<?, ? examples/s]

In [6]:
training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=8,
    # evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
)

trainer.train()

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


Step,Training Loss


TrainOutput(global_step=95, training_loss=0.2906909942626953, metrics={'train_runtime': 155.6202, 'train_samples_per_second': 4.787, 'train_steps_per_second': 0.61, 'total_flos': 196051175562240.0, 'train_loss': 0.2906909942626953, 'epoch': 5.0})

In [7]:
model.save_pretrained("./my_model")
tokenizer.save_pretrained("./my_model")

('./my_model/tokenizer_config.json',
 './my_model/special_tokens_map.json',
 './my_model/vocab.txt',
 './my_model/added_tokens.json')

In [11]:
def predict(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding="max_length", max_length=512)
    inputs = {k: v.to(model.device) for k, v in inputs.items()}
    with torch.no_grad():
        outputs = model(**inputs)
        preds = torch.sigmoid(outputs.logits)
        preds = preds.detach().cpu().numpy()[0]
    result = [labels_list[i] for i, p in enumerate(preds) if p >= 0.5]  # порог 0.5, можно менять
    return result

In [12]:
df_big = df_150

df_big['predicted_mitigating_factors'] = df_big['result_text'].apply(predict)
df_big.to_csv("predicted_mf_150.csv", index=False)

In [16]:
def normalize_factors(factor_str):
    if isinstance(factor_str, str):
        factors = [f.strip() for f in factor_str.split(';')]
    elif isinstance(factor_str, list):
        factors = factor_str
    else:
        return []
    return sorted(list(set(factors)))

In [17]:
df_big['mitigating_factors_standardized'] = df_big['mitigating_factors_standardized'].apply(normalize_factors)
df_big['predicted_mitigating_factors'] = df_big['predicted_mitigating_factors'].apply(normalize_factors)

In [19]:
df_big.to_csv("predicted_mf_150_2.csv", index=False)

In [23]:
df_big[['mitigating_factors_standardized', 'predicted_mitigating_factors']].head(10)

,mitigating_factors_standardized,predicted_mitigating_factors
0,"[активное содействие следствию, признание вины...","[признание вины, раскаяние]"
1,"[активное содействие следствию, наличие несове...","[признание вины, раскаяние]"
2,"[активное содействие следствию, наличие несове...","[признание вины, раскаяние]"
3,"[активное содействие следствию, наличие несове...","[признание вины, раскаяние]"
4,"[активное содействие следствию, наличие несове...","[признание вины, раскаяние]"
5,"[активное содействие следствию, наличие несове...","[признание вины, раскаяние]"
6,"[активное содействие следствию, наличие несове...","[признание вины, раскаяние]"
7,"[активное содействие следствию, наличие иждиве...","[признание вины, раскаяние]"
8,"[активное содействие следствию, положительная ...","[признание вины, раскаяние]"
9,"[активное содействие следствию, наличие несове...","[признание вины, раскаяние]"


In [28]:
# # df_train = df_150
# excluded = {'признание вины', 'раскаяние'}

# def filter_factors(factors):
#     if isinstance(factors, str):
#         factors = [f.strip() for f in factors.split(';')]
#     return [f for f in factors if f not in excluded]

# # df_train['factors_filtered'] = df_train['mitigating_factors_standardized'].apply(filter_factors)
# df_big['pred_filtered'] = df_big['predicted_mitigating_factors'].apply(filter_factors)
# df_big['true_filtered'] = df_big['mitigating_factors_standardized'].apply(filter_factors)


# dataset = Dataset.from_pandas(df_train)
# dataset = dataset.map(tokenize_and_encode, remove_columns=dataset.column_names)

In [ ]:
# 0k to 1k
big_df = df_0k1k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_0k1k.csv", index=False)

predicted_result_type_df_0k1k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_0k1k.to_csv("predicted_mit_factors_1k0k.csv", index=False)

In [ ]:
# 1k to 2k
big_df = df_1k2k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_1k2k.csv", index=False)

predicted_result_type_df_1k2k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_1k2k.to_csv("predicted_mit_factors_1k2k.csv", index=False)

In [ ]:
# 2k to 3k
big_df = df_2k3k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_2k3k.csv", index=False)

predicted_result_type_df_2k3k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_2k3k.to_csv("predicted_mit_factors_2k3k.csv", index=False)

In [ ]:
# 3k to 4k
big_df = df_3k4k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_3k4k.csv", index=False)

predicted_result_type_df_3k4k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_3k4k.to_csv("predicted_mit_factors_3k4k.csv", index=False)

In [ ]:
# 4k to 5k
big_df = df_4k5k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_4k5k.csv", index=False)

predicted_result_type_df_4k5k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_4k5k.to_csv("predicted_mit_factors_4k5k.csv", index=False)

In [ ]:
# 5k to 6k
big_df = df_5k6k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_5k6k.csv", index=False)

predicted_result_type_df_5k6k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_5k6k.to_csv("predicted_mit_factors_5k6k.csv", index=False)

In [ ]:
# 6k to 7k
big_df = df_6k7k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_6k7k.csv", index=False)

predicted_result_type_df_6k7k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_6k7k.to_csv("predicted_mit_factors_6k7k.csv", index=False)

In [ ]:
# 7k to 8k
big_df = df_7k8k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_7k8k.csv", index=False)

predicted_result_type_df_7k8k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_7k8k.to_csv("predicted_mit_factors_7k8k.csv", index=False)

In [ ]:
# 8k to 9k
big_df = df_8k9k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_8k9k.csv", index=False)

predicted_result_type_df_8k9k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_8k9k.to_csv("predicted_mit_factors_8k9k.csv", index=False)

In [ ]:
# 9k to 10k
big_df = df_9k10k
big_df['predicted_mitigating_factors'] = big_df['result_text'].apply(predict_result_type)
big_df.to_csv("predicted_mf_df_9k10k.csv", index=False)

predicted_result_type_df_9k10k = big_df[['id', 'predicted_result_type']]
predicted_result_type_df_9k10k.to_csv("predicted_mit_factors_9k10k.csv", index=False)

# расчёт метрик

In [29]:
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics import accuracy_score, f1_score, classification_report

# 1. Binarizer
mlb = MultiLabelBinarizer(classes=labels_list)

# 2. Преобразуем оба столбца
y_true = mlb.fit_transform(df_big['mitigating_factors_standardized'])
y_pred = mlb.transform(df_big['predicted_mitigating_factors'])

# 3. Accuracy и F1 (macro — среднее по классам)
accuracy = (y_true == y_pred).mean()  # Simple sample-level accuracy
f1 = f1_score(y_true, y_pred, average='macro')

# 4. Вывод
print(f"Accuracy: {accuracy:.4f}")
print(f"Macro F1-score: {f1:.4f}")
print(classification_report(y_true, y_pred, target_names=labels_list))


Accuracy: 0.9255
Macro F1-score: 0.0892
                                                                 precision    recall  f1-score   support

                                                 признание вины       0.93      1.00      0.97       139
                                                      раскаяние       0.83      1.00      0.91       124
               наличие несовершеннолетнего ребенка на иждивении       0.00      0.00      0.00        45
                                             состояние здоровья       0.00      0.00      0.00        53
                                   положительная характеристика       0.00      0.00      0.00        29
                              удовлетворительная характеристика       0.00      0.00      0.00         2
активное способствование раскрытию и расследованию преступления       0.00      0.00      0.00         0
                                                явка с повинной       0.00      0.00      0.00        31
              

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:909: UserWarning: unknown class(es) ['активно способствовал раскрытию и расследованию преступления', 'активное содействие следствию', 'активное содействие следствию и расследованию преступлений', 'активное содействие следствию и расследованию преступленияо месту жительства характеризуется удовлетворительно', 'активное содействие следствию и расследованию преступленияэ', 'активное способствование раскрытию и расследованию преступления преступления', 'беременность супруги', 'наличие заболеваний', 'наличие иждивенца', 'наличие инвалидности', 'наличие совершеннолетнего ребенка на иждивении', 'положительная характеристика по месту жительства', 'положительная характеристику её личности', 'поощрения в виде грамот', 'ранение в сво', 'состояние его здоровья', 'участие в сво'] will be ignored
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: F-score is i